# Exercise 05:  Solving differential equation-based Bayesian inverse problems using CUQIpy

Here we build a Bayesian problem in which the forward model is a partial differential equation model, 1D Heat problem in particular.

**Try to at least run through part 1 to 4 before working on the optional exercises**

## Learning objectives of this notebook:
- Solve PDE-based Bayesian problem using CUQIpy.
- Parametrization of the Bayesian parameters (e.g. KL expansion, non-linear maps).
- Introducing CUQIpy's PDE class.

## Table of contents: 
* [1. Loading the PDE test problem](#PDE_model)
* [2. Building and solving the Bayesian inverse problem](#inverse_problem)
* [3. Parametrizing the Bayesian parameters via a general mapping  to enforce positivity](#mapped_geometries)
* [4. Parametrizing the Bayesian parameters via step function expansion](#step_function)
* [5. Parametrizing the Bayesian parameters via KL expansion](#KL_expansion) ★
* [6. elaboration: the PDEmodel class](#PDE_model_elaborate) ★


##  1. Loading the PDE test problem <a class="anchor" id="PDE_model"></a>

We first import the required python standard packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor

From cuqi package, we import the classes that we use in this exercise

In [ ]:
from cuqi.geometry import MappedGeometry, KLExpansion
from cuqi.pde import SteadyStateLinearPDE
from cuqi.distribution import GaussianCov, Posterior, Gaussian
from cuqi.sampler import pCN
from cuqi.testproblem import Heat_1D
from cuqi.operator import FirstOrderFiniteDifference
from cuqi.pde import SteadyStateLinearPDE

We load the test problem `Heat_1D` which provides a one dimensional (1D) time dependent heat model with zero boundary conditions. The model is discretized using finite difference.

The PDE is given by:

$$ \frac{\partial u(x,t)}{\partial t} - c^2 \Delta_x u(x,t)   = f(x,t), \;\text{in}\;\Omega=[0,L] $$
$$u(0,t)= u(L,t)= 0 $$

where $u(x,t)$ is the temperature and $c^2$ is the thermal diffusivity. We assume the source term $f$ is zero. The unknown Bayesian parameters for this test problem is the initial heat profile $\theta(x):=u(x,0)$. The data $\mathbf{d}$ are the temperature measurements everywhere in the domain at the final time $T$.

We load `Heat_1D` using `get_components` method. We can explore the initialization parameters (and hence what can be passed to `get_components` method) of the `Heat_1D` test problem by calling `Heat_1D?`. We choose the following set up for the test problem: Number of finite difference nodes N, length of domain L, and the final time T.

In [ ]:
N = 20  # number of finite difference nodes            
L = 1    # Length of the domain
T = 0.05  # Final time

We choose the initial condition (the exact solution for the Bayesian problem) to be a step function with three pieces.

In [ ]:
myExactSolution = np.zeros(N)
myExactSolution[:floor(N/3)] = 1
myExactSolution[floor(N/3):floor(2*N/3)] = 2
myExactSolution[floor(2*N/3):] = 3

And now we load the `Heat_1D` problem providing our own exact solution:

In [ ]:
model, data, problemInfo = Heat_1D.get_components(dim=N, endpoint=L, max_time=T, exactSolution=myExactSolution)

Lets take a look at what we obtain from the test problem. We view the `model`:

In [ ]:
model

We can look at the data:

In [ ]:
data

And the `problemInfo`:

In [ ]:
problemInfo

Now lets plot the exact solution of this inverse problem and the exact and noisy data

In [ ]:
problemInfo.exactSolution.plot()
problemInfo.exactData.plot()
data.plot()
plt.legend(['exact solution', 'exact data', 'noisy data']);

Note that the boundaries at 0 and $L$ are not included in this plot.

## 2. Building and solving the Bayesian inverse problem <a class="anchor" id="inverse_problem"></a>

Here we want to define the prior, the likelihood and the posterior distribution. We start by defining a simple Gaussian prior.

In [ ]:
mean = 0
std = 1.2
prior = Gaussian(mean*np.ones(N), std, geometry= model.domain_geometry)


#### Try yourself (optional)
* create prior samples (~1 line).
* plot the 95% credibility interval of the prior samples (~1 line).
* look at the 95% credibility interval of the PDE model solution to quantify the forward uncertainty (~2 lines).


In [ ]:
# Your code here



We then set up the likelihood. We obtain information about the noise distribution from `problemInfo.infoString`:

In [ ]:
problemInfo.infoString

We define the likelihood:

In [ ]:
SNR = 200
sigma_likelihood = np.linalg.norm(problemInfo.exactData)/SNR
likelihood = Gaussian(mean=model, std=sigma_likelihood, corrmat=np.eye(N), geometry=model.range_geometry).to_likelihood(data)

Now that we have all the components we need, we can create the posterior distribution:

In [ ]:
posterior =  Posterior(likelihood, prior)

We can now sample the posterior. Lets try the preconditioned Crank-Nicolson (pCN) sampler (for now we fix the random seed for simplification, different seeds lead to different results, but in principle, if the sample size is large enough, the seed choice should be inconsequential):

In [ ]:
np.random.seed(0)
MySampler = pCN(posterior)
posterior_samples = MySampler.sample_adapt(5000)

Let's look at the samples:

In [ ]:
posterior_samples.plot_ci(95, exact = problemInfo.exactSolution)

We can also look at chains of particular parameters, for example, lets look at the initial condition estimation at first and second nodes:

In [ ]:
posterior_samples.plot_chain([0,1]);

We note that the chains have a notable correlation. This limits the number of independent samples that we obtain from this chain.  

## 3. Parametrizing the Bayesian parameters via a general mapping to enforce positivity <a class="anchor" id="mapped_geometries"></a> 

Here we introduce the concept of mapped geometries. In many inverse problems, parametrization of the forward model input through possibly nonlinear functions might be needed. For example, in this 1D heat example, lets assume that we want to enforce positivity of the initial condition $u(x,0) =\theta(x)$. We can use the parametrization $u(x,0) = e^{\theta(x)}$, where $\theta$ is the Bayesian parameters (log initial condition).  

In `CUQIpy`, this can be achieved through a `MappedGeometry` object. Lets update the exact solution, and the domain geometry and test this idea:  

In [ ]:
model, data, problemInfo = Heat_1D.get_components(dim=N, endpoint=L, max_time=T, exactSolution = myExactSolution)
model.domain_geometry = MappedGeometry(geometry = model.domain_geometry, map= lambda x : np.exp(x))
prior = Gaussian(mean*np.ones(N), std, geometry= model.domain_geometry)
likelihood = Gaussian(mean=model, std=sigma_likelihood, corrmat=np.eye(N), geometry=model.range_geometry).to_likelihood(data)
posterior =  Posterior(likelihood, prior)
MySampler = pCN(posterior)
np.random.seed(0)
posterior_samples = MySampler.sample_adapt(5000)

In [ ]:
posterior_samples.plot_ci(95, exact = problemInfo.exactSolution)

The parameterization `map= lambda x : np.exp(x)` ensures that the Bayesian parameters are indeed positive. But beyond that not much is improved.

## 4. Parametrizing the Bayesian parameters via step function expansion <a class="anchor" id=" step_function"></a> 

One way to improve the solution of this Bayesian problem is to use better prior information. Here we assume the prior is a step function with three pieces. This also makes the Bayesian problem simpler because now we only have three Bayesian parameters to infer.

To do this in this test case we pass `field_type='Step'` to `Heat_1D.get_components`, which changes the domain geometry during creation of the model.

In [ ]:
np.random.seed(0)
N=20
model, data, problemInfo = Heat_1D.get_components(dim=N, endpoint=L, max_time=T,field_type='Step')

Lets look at the model.domain_geometry in this case: 

In [ ]:
model.domain_geometry

We then continue to create the Bayesian problem with a prior of dimension = 3. 

In [ ]:
prior = Gaussian(mean*np.ones(3), std, geometry= model.domain_geometry)
likelihood = Gaussian(mean=model, std=sigma_likelihood, corrmat=np.eye(N), geometry=model.range_geometry).to_likelihood(data)
posterior =  Posterior(likelihood, prior)
MySampler = pCN(posterior)
posterior_samples = MySampler.sample_adapt(5000)

Let's take a look at the posterior:

In [ ]:
posterior_samples.plot_ci(95, exact = problemInfo.exactSolution)
posterior_samples.shape

#### Try it yourself (optional):
* For this step function parametrization, try to enforce positivity of the posterior samples via the `MappedGeometry` and run the pCN sampler again (similar to part 3).

In [ ]:
# Your code here



## 5 Parametrizing the Bayesian parameters via KL expansion ★

Here we explore the Bayesian inversion for a more general exact solution. We parametrize the Bayesian parameters using Karhunen–Loève (KL) expansion. This will represent the initial heat profile as a linear combination of sine functions. 
$$ u(x,0) = \sum_i \theta_i  (1/i)^{\text{decay}}  sin(\frac{i L x}{\pi}). $$
Where $\theta_i$ are the Bayesian parameters. 

For this test case we also define a prior correlation matrix from the following correlation function:

In [ ]:
var = 10
lc = 0.2
p = 2
C_YY = lambda x1, x2: var*np.exp( -(1/p) * (abs( x1 - x2 )/lc)**p )

Lets load the Heat_ID test case and pass `field_type = 'KL'`, which behind the scenes will modify the domain geometry of the model to a KL expansion:

In [ ]:
model, data, problemInfo = Heat_1D.get_components(dim=N, endpoint=L, max_time=T, field_type = 'KL' )

Now we inspect the `model.domain_geometry`:

In [ ]:
model.domain_geometry

And the exact solution and the data:

In [ ]:
problemInfo.exactSolution.plot()
problemInfo.exactData.plot()
data.plot()
plt.legend(['exact solution', 'exact data', 'noisy data']);

We discretize the correlation function and define the prior, the likelihood and the posterior:

In [ ]:
x = model.domain_geometry.grid
XX, YY = np.meshgrid(x, x, indexing='ij')
sigma_prior = C_YY(XX, YY)
prior = GaussianCov(mean*np.ones(N), sigma_prior, geometry= model.domain_geometry)
likelihood = Gaussian(mean=model, std=sigma_likelihood, corrmat=np.eye(N), geometry=model.range_geometry).to_likelihood(data)
posterior =  Posterior(likelihood, prior)

We sample the posterior:

In [ ]:
np.random.seed(0)
MySampler = pCN(posterior)
posterior_samples = MySampler.sample_adapt(5000)

And plot the $95\%$ credibility interval (you can try plotting difference credibility intervals) 

In [ ]:
posterior_samples.plot_ci(95, exact = problemInfo.exactSolution)

We see that the exact solution is contained within the credibility interval. We can look at the expansion parameters credibility intervals in both the prior and posterior distributions by passing `plot_par=True` to `plot_ci` function:

In [ ]:
posterior_samples.plot_ci(100, plot_par=True)
plt.xticks(np.arange(prior.dim)[::5]);
plt.figure()
prior.sample(1000).plot_ci(100, plot_par=True)
plt.xticks(np.arange(prior.dim)[::5]);

We note that in the posterior, in general, smoother modes ($v0-v4$) are inferred with higher certainty than oscillatory modes. 

## 6. Elaboration: the PDEmodel class <a class="anchor" id="PDE_model_elaborate"></a> ★

Lets explore the model for PDE problems.

#### Try it yourself (optional):

* View: `model`, `model.pde`, `model.pde.PDE_form`

We can, for example, create our own PDE model for simple Poisson equation with zero boundaries. We first create the forward difference operator using the cuqi operator `FirstOrderFiniteDifference`.

In [ ]:
n_poisson = 1000 #Number of nodes
L = 1 # Length of the domain
dx = L/(n_poisson-1) # grid spacing
diff_operator = FirstOrderFiniteDifference(n_poisson,bc_type='zero').get_matrix().todense()/dx

We then construct the source term (point source):

In [ ]:
source_term = np.zeros(n_poisson)
source_term[int(n_poisson/2)] = 1/dx 

We create the PDE form which consists of the differential operator and the right hand side, and is a function of the Bayesian parameter x. 

In [ ]:
poisson_form = lambda x: (diff_operator.T@diff_operator, x* source_term)

We create the CUQI PDEModel, in this case a `SteadyStateLinearPDE` model.

In [ ]:
CUQI_pde = SteadyStateLinearPDE(poisson_form)

The model `CUQI_pde` has three main methods: 

1. assemble, which assembles the differential operator and the RHS given the Bayesian parameter x.
2. solve, which solves the PDE.
3. observe, for now observe returns the solution of the PDE, but it is to be generalized to apply observation operators on the PDE solution (e.g. extracting final temperature at specific or random points).

In the following we assemble and solve this Poisson problem.

In [ ]:
CUQI_pde.assemble(5)
sol, info = CUQI_pde.solve()

And plot the solution:

In [ ]:
plt.plot(np.linspace(dx,L,n_poisson,endpoint=False),sol)

#### Try it yourself (optional):

* Double the magnitude of the source term by editing the line `CUQI_pde.assemble(5)` above. Look at the solution.

In [ ]:
# Your code here

